In [18]:
with open("dry_run_3.log") as f:
    text = f.read()

In [19]:
text

'# building\nmake -C phantom vmonly\n# building\nmake -C gl\n# FULL_FILE_PATH := specbuf.c\necho ---- cc {"target" : "specbuf.o", "file" : "specbuf.c" , "paths" : "/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl"}\ngcc -c -o specbuf.o -W -Wall -Wno-unused-parameter -Wno-sign-compare -m64 -O0 -g -MD  -I/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/spec/x86_64/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode_libc/spec/x86_64/libc -I/home/anton/Sandbox/PhantomOS/phantomuserland/include -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode-amd64 -I/home/anton/Sandbox/PhantomOS/phantomuserland/include -I/home/anton/Sandbox/PhantomOS/phantomuserland/include/genode-amd64 -nostdinc -std=gnu89 -DKERNEL -DARCH_genode-amd64=1 -DBOARD_amd64

In [20]:
import re

sections_raw = re.split(r"make -C (.*?)\n", text)

In [21]:
cc_lines = [x for x in text.split("\n") if x.startswith("echo ---- cc ")]
# print(cc_lines)

In [22]:
import json

cc_lines = [x[len("echo ---- cc "):] for x in cc_lines]

In [23]:
cc_lines = [x.strip() for x in cc_lines]
targets = [json.loads(x) for x in cc_lines]

In [24]:
len(targets)

274

In [25]:
sections = list(zip(sections_raw[1::2], sections_raw[2::2]))

In [26]:
sections[0]

('phantom vmonly', '# building\n')

In [27]:
# # for s in section:
# s = sections[0]
# cc_lines = [x for x in s[1].split("\n") if x.startswith("echo ---- cc ")]
# cc_lines = [x[len("echo ---- cc "):] for x in cc_lines]
# targets = [json.loads(x) for x in cc_lines]

In [28]:
# import os

# valid_paths = []

# for t in targets:
#     paths = [os.path.join(s, t["file"]) for s in t["paths"].split(" ")]
#     valid_paths.append((paths, [os.path.exists(x) for x in paths]))



In [29]:
# # Simple check
# for v in valid_paths:
#     print(v)
#     if v[1].count(True) != 1:
#         print("------ " + str(v[1].count(True)))
#         print(v[0])

In [30]:
import os

files_per_section = []

for s in sections:
    cc_lines = [x for x in s[1].split("\n") if x.startswith("echo ---- cc ")]
    cc_lines = [x[len("echo ---- cc "):] for x in cc_lines]
    targets = [json.loads(x) for x in cc_lines]
    
    valid_paths = []

    for t in targets:
        paths = list(set([os.path.join(s, t["file"]) for s in t["paths"].split(" ")]))
        valid_paths.append((paths, [os.path.exists(x) for x in paths]))
    
    current_files_list = []
    
    # Simple 
    print("######  " + str(s[0]))
    for v in valid_paths:
#         print(v)
        if v[1].count(True) != 1:
            # Something went wrong if you are in this block
            print("------ " + str(v[1].count(True)))
#             print(v[0])
            print("Trying using set()")
            unique_values = set(v[0])
            if (len(unique_values) != 1):
                print("Didn't work. Values={}".format(v[0]))
            else:
                print("Worked :)")
        else:
            current_files_list.append(v[0][v[1].index(True)])
    
    files_per_section.append((s[0], current_files_list))

######  phantom vmonly
######  gl
######  libtuned
######  libphantom
######  libwin
######  libfreetype
######  libc
######  vm pvm_headless


In [31]:
files_per_section

[('phantom vmonly', []),
 ('gl',
  ['/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/specbuf.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/phantom_loop.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/list.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/error.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/texture.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/select.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/init.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/light.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/arrays.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/vertex.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/oscontext.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/image_util.c',
   '/home/anton/Sandbox/PhantomOS/phantomuserland/phantom/gl/clip.c',
   '/home/anton/Sandbox/

In [32]:
# Ok, now need to normalize and transform paths. so we can use them in Genode

In [33]:
PHANTOM_HOME = "/home/anton/Sandbox/PhantomOS/phantomuserland"

norm_files_per_section = [(x[0], [os.path.relpath(os.path.normpath(t), PHANTOM_HOME) for t in x[1]]) for x in files_per_section]

In [34]:
norm_files_per_section

[('phantom vmonly', []),
 ('gl',
  ['phantom/gl/specbuf.c',
   'phantom/gl/phantom_loop.c',
   'phantom/gl/list.c',
   'phantom/gl/error.c',
   'phantom/gl/texture.c',
   'phantom/gl/select.c',
   'phantom/gl/init.c',
   'phantom/gl/light.c',
   'phantom/gl/arrays.c',
   'phantom/gl/vertex.c',
   'phantom/gl/oscontext.c',
   'phantom/gl/image_util.c',
   'phantom/gl/clip.c',
   'phantom/gl/phantomdrvr.c',
   'phantom/gl/api.c',
   'phantom/gl/misc.c',
   'phantom/gl/memory.c',
   'phantom/gl/clear.c',
   'phantom/gl/zdither.c',
   'phantom/gl/msghandling.c',
   'phantom/gl/ztriangle.c',
   'phantom/gl/zline.c',
   'phantom/gl/zmath.c',
   'phantom/gl/zbuffer.c',
   'phantom/gl/matrix.c',
   'phantom/gl/texobj.c',
   'phantom/gl/get.c']),
 ('libtuned', []),
 ('libphantom',
  ['phantom/libphantom/console_color.c',
   'phantom/libphantom/wired_spin.c',
   'phantom/libphantom/udp_json_builder.c',
   'phantom/libphantom/atoin.c',
   'phantom/libphantom/init.c',
   'phantom/libphantom/i_disk

In [35]:
for x in norm_files_per_section:
    print("---------- " + x[0])
    for f in x[1]:
        print("{} \\".format(f))

---------- phantom vmonly
---------- gl
phantom/gl/specbuf.c \
phantom/gl/phantom_loop.c \
phantom/gl/list.c \
phantom/gl/error.c \
phantom/gl/texture.c \
phantom/gl/select.c \
phantom/gl/init.c \
phantom/gl/light.c \
phantom/gl/arrays.c \
phantom/gl/vertex.c \
phantom/gl/oscontext.c \
phantom/gl/image_util.c \
phantom/gl/clip.c \
phantom/gl/phantomdrvr.c \
phantom/gl/api.c \
phantom/gl/misc.c \
phantom/gl/memory.c \
phantom/gl/clear.c \
phantom/gl/zdither.c \
phantom/gl/msghandling.c \
phantom/gl/ztriangle.c \
phantom/gl/zline.c \
phantom/gl/zmath.c \
phantom/gl/zbuffer.c \
phantom/gl/matrix.c \
phantom/gl/texobj.c \
phantom/gl/get.c \
---------- libtuned
---------- libphantom
phantom/libphantom/console_color.c \
phantom/libphantom/wired_spin.c \
phantom/libphantom/udp_json_builder.c \
phantom/libphantom/atoin.c \
phantom/libphantom/init.c \
phantom/libphantom/i_disk.c \
phantom/libphantom/hashfunc.c \
phantom/libphantom/udp_json.c \
phantom/libphantom/pool.c \
phantom/libphantom/hexd

In [36]:
for x in norm_files_per_section:
    print("---------- " + x[0])

---------- phantom vmonly
---------- gl
---------- libtuned
---------- libphantom
---------- libwin
---------- libfreetype
---------- libc
---------- vm pvm_headless
